# Unsupervised Handwritten Signature Verification System Using Extreme Learning Machine


## Overview 


Unsupervised signature verification attempts to tell a forged signature from a real one based on the signature features alone. This idea is inspired by the existence of a generative model for genuine and forged signatures. This model generates a handwriting process for creating a genuine signature from a sequence of characters and generates an imitation process that omits some signature features for creating a skilled forgery of that signature. A method that could reverse this generative model would tell a genuine signature from a forged one without the need for ground truth training data samples

## Preprocessing

In [3]:
# Importing libraries 
import tensorflow as tf
import numpy as np
import cv2
import pandas as pd


In [2]:
pip install sklearn_extensions

  Created wheel for sklearn-extensions: filename=sklearn_extensions-0.0.2-py2.py3-none-any.whl size=24578 sha256=5a9c4a43d5770153ecf9301b9b0d72bfdb12b0180b67a4fafaa5399088144338
  Stored in directory: c:\users\micky darlyn\appdata\local\pip\cache\wheels\5b\4f\12\e56caf24d4ce8e90d3734238d9307c12b1d6c1a211889d85a4
Successfully built sklearn-extensions
Note: you may need to restart the kernel to use updated packages.


ModuleNotFoundError: No module named 'sklearn_extensions'

In [3]:
import os

### Getting signature images into workspace 


In [37]:
#extrating the genuine and forged image paths tor test 

test_genuine_image_paths = []
test_forged_image_paths = []

# for genuine
for i in range (1,151):
    directroy_path = f"./New folder (10)/test/{i}/genuine/"    
    for file in os.listdir(directroy_path):
        test_genuine_image_paths.append(file)
        
        
# for forged 
for i in range (1,151):
    directroy_path = f"./New folder (10)/test/{i}/forge/"    
    for file in os.listdir(directroy_path):
        test_forged_image_paths.append(file)
    


In [38]:
print(len(test_genuine_image_paths))
print(len(test_forged_image_paths))

1200
2100


In [40]:
#extrating the genuine and forged image paths tor train 

train_genuine_image_paths = []
train_forged_image_paths = []

# for genuine
for i in range (1,151):
    directroy_path = f"./New folder (10)/train/genuine/"    
    for file in os.listdir(directroy_path):
        train_genuine_image_paths.append(file)
        
        
# for forged 
for i in range (1,151):
    directroy_path = f"./New folder (10)/train/forge/"    
    for file in os.listdir(directroy_path):
        train_forged_image_paths.append(file)
    


In [41]:
training_image_paths = []
print(len(train_genuine_image_paths))
print(len(train_forged_image_paths))

360000
360000


### Feature Extraction 

In [ ]:

from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from skimage.util import view_as_windows

In [ ]:
# Feature Extration using the inceptionV3 model

def extract_features(image, model):
    # Resize the image to 299x299 to match the input size of the InceptionV3 model
    image = tf.image.resize(image, (299, 299))
    
    # Preprocess the image for input to the InceptionV3 model
    image = preprocess_input(image)
    
    # Use the InceptionV3 model to extract features from the image
    features = model.predict(np.array([image]))
    
    # Flatten the features into a 1D array
    features = features.flatten()
    
    return features

In [ ]:
# list to hold all image features 
signature_features = []


for path in training_image_paths:
    feature = extract_features(path,InceptionV3)
    signature_features.append(feature)
    
X = np.array(signature_features)

In [4]:
from sklearn_extensions.extreme_learning_machines.random_layer import RBFSampler
from sklearn_extensions.extreme_learning_machines.elm import ELMRegressor
from sklearn.cluster import KMeans

C:\Users\Micky Darlyn\anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass neg_label=-1, pos_label=1 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


ImportError: cannot import name 'RBFSampler' from 'sklearn_extensions.extreme_learning_machines.random_layer' (C:\Users\Micky Darlyn\anaconda3\lib\site-packages\sklearn_extensions\extreme_learning_machines\random_layer.py)

In [ ]:
class UnsupervisedELM:
    def __init__(self, density=0.01, l2_regularization=0.01, min_neurons=2, max_neurons=32768):
        self.density = density
        self.l2_regularization = l2_regularization
        self.min_neurons = min_neurons
        self.max_neurons = max_neurons
        self.elm_model = None

    def fit(self, X):
        num_features = X.shape[1]
        num_neurons_list = np.logspace(np.log2(self.min_neurons), np.log2(self.max_neurons), num=20, base=2, dtype=int)
        rp_layer = RBFSampler(gamma=1.0 / num_features, n_components=num_neurons_list[0], random_state=0, density=self.density)
        rp_layer.fit(X)
        hidden_output = rp_layer.transform(X)
        for n in num_neurons_list[1:]:
            rp_layer = RBFSampler(gamma=1.0 / num_features, n_components=n, random_state=0, density=self.density)
            rp_layer.fit(X)
            hidden_output = np.concatenate([hidden_output, rp_layer.transform(X)], axis=1)
            kmeans = KMeans(n_clusters=n, random_state=0).fit(hidden_output)
            hidden_output = kmeans.transform(hidden_output)
        self.elm_model = ELMRegressor(n_hidden=num_neurons_list[-1], alpha=self.l2_regularization)
        self.elm_model.fit(hidden_output)
        
    def predict(self, X):
        hidden_output = self.elm_model.random_layer.transform(X)
        for kmeans in self.elm_model.hidden_layer:
            hidden_output = kmeans.transform(hidden_output)
        return self.elm_model.output_layer.predict(hidden_output)
